# import

In [1]:
import torch.nn as nn
import torch
import chess
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from model_architecture.neural_network import NNUE

# Encoding functions
pieces = list('rnbqkpRNBQKP.')
def one_hot_encode_piece(piece):
    arr = np.zeros(len(pieces))
    piece_to_index = {p: i for i, p in enumerate(pieces)}    
    index = piece_to_index[piece]
    arr[index] = 1
    return arr

def encode_board(board):
    board_str = str(board).replace(' ', '')
    board_list = []
    for row in board_str.split('\n'):
        for piece in row:
            board_list.append(one_hot_encode_piece(piece))
    return np.array(board_list)

def encode_fen_string(fen_str):
    board = chess.Board(fen=fen_str)
    return encode_board(board)

# load data

In [2]:
# Load data
df = pd.read_csv('../chess-data/fen/train.csv')
print('original shape:',df.shape)
df = df[:1000]
print('used shape:', df.shape)

original shape: (1979383, 2)
used shape: (1000, 2)


# transform data

In [3]:
df['FEN']

0      r1b2rk1/ppp2pbp/3q1np1/n3p1B1/2B5/1Q3N2/PP1N1P...
1      8/1pp2p2/6k1/4P2p/p1PR1K1P/2r2P2/6P1/8 w - - 0 33
2      r2qk1nr/1b3pbp/n3p1p1/1pp1P3/p2PN3/2P2N2/PPB3P...
3      2b2rk1/5pp1/p2q1n1p/P2pn3/3N4/3BP1B1/2Q2PPP/Rr...
4      r2qkb1r/ppp2ppb/2n1p3/3n2PQ/3Pp3/2P4P/PP6/RNB1...
                             ...                        
995    rnbqk1nr/pppp1ppp/4p3/4P3/1b1P1P2/2P5/PP4PP/RN...
996    rR4k1/2p1qp1p/2b1pb2/2Pp2p1/p2P1B2/P1Q1PN2/5PP...
997             8/pppnk3/8/7P/8/8/P1P3P1/2KR4 w - - 0 30
998    rnbqkb1r/ppp3pp/4pn2/3p1pB1/3PP3/2N2P2/PPP3PP/...
999    r1bq1rk1/pp3ppp/5n2/4p3/3n4/1QNP1P2/P3B1PP/R3K...
Name: FEN, Length: 1000, dtype: object

In [4]:
# Encode data
print('Encoding data...')
st = time.time()
df['FEN'] = df['FEN'].apply(encode_fen_string)
print(f'Time elapsed: {round(time.time()-st)}s')

Encoding data...
Time elapsed: 0s


In [5]:
df['FEN']

0      [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1      [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
2      [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
3      [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
4      [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
                             ...                        
995    [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
996    [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
997    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
998    [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
999    [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
Name: FEN, Length: 1000, dtype: object

# split data and covert tensor for traning

In [6]:
X = np.stack(df['FEN'].values)  # Convert list of arrays to a single numpy array
y = df['Evaluation'].values
# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)  # Shape: (1000, 64, 13)
y = torch.tensor(y, dtype=torch.float32)  # Shape: (1000,)
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify shapes
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

X_train shape: torch.Size([800, 64, 13])
X_test shape: torch.Size([200, 64, 13])
y_train shape: torch.Size([800])
y_test shape: torch.Size([200])


# setup model

In [17]:
model = NNUE()
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer
model

NNUE(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_tanh_stack): Sequential(
    (0): Linear(in_features=832, out_features=256, bias=True)
    (1): Tanh()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=8, bias=True)
    (5): Tanh()
  )
  (output): Linear(in_features=8, out_features=1, bias=True)
)

# training and save model

In [8]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for inputs, targets in train_loader:
        inputs, targets = inputs, targets
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))  # Reshape targets to match output
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Print epoch loss
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Save model
torch.save(model.state_dict(), "./checkpoint/nnue_1000data_50epochs.pth")

Epoch [1/50], Loss: 311926.6219
Epoch [2/50], Loss: 311865.4000
Epoch [3/50], Loss: 311854.3044
Epoch [4/50], Loss: 311845.4650
Epoch [5/50], Loss: 311732.7325
Epoch [6/50], Loss: 311297.1653
Epoch [7/50], Loss: 310955.4009
Epoch [8/50], Loss: 310814.5400
Epoch [9/50], Loss: 310562.4194
Epoch [10/50], Loss: 310338.6353
Epoch [11/50], Loss: 310160.1389
Epoch [12/50], Loss: 309951.8178
Epoch [13/50], Loss: 309769.5081
Epoch [14/50], Loss: 309830.1909
Epoch [15/50], Loss: 309534.5225
Epoch [16/50], Loss: 309403.4875
Epoch [17/50], Loss: 309190.6462
Epoch [18/50], Loss: 309023.6997
Epoch [19/50], Loss: 308859.4398
Epoch [20/50], Loss: 308711.4756
Epoch [21/50], Loss: 308572.2019
Epoch [22/50], Loss: 308436.0381
Epoch [23/50], Loss: 308314.9406
Epoch [24/50], Loss: 308175.7459
Epoch [25/50], Loss: 308040.6097
Epoch [26/50], Loss: 307916.2372
Epoch [27/50], Loss: 307860.3422
Epoch [28/50], Loss: 308172.4678
Epoch [29/50], Loss: 307861.3513
Epoch [30/50], Loss: 307765.4919
Epoch [31/50], Loss

# evaluation

In [12]:
model.eval()  # Set model to evaluation mode
test_loss = 0.0

with torch.no_grad():
    for inputs, targets in test_loader:
        print(inputs.shape)
        inputs, targets = inputs, targets
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
print(f"Test Loss: {avg_test_loss:.4f}")

torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 64, 13])
torch.Size([1, 6

# minimax + nnue

In [ ]:
new_model = NNUE()
new_model.load_state_dict(torch.load('./checkpoint/nnue_1000data_50epochs.pth', weights_only=True))
new_model.eval()

fen = '8/1pp2p2/6k1/4P2p/p1PR1K1P/2r2P2/6P1/8 w - - 0 33'
fen = np.stack(encode_fen_string(fen))
fen = torch.tensor(fen, dtype=torch.float32).unsqueeze(0)
fen.shape

In [30]:
import chess
import torch
    
class MinimaxNNUE:
    def __init__(self, depth=3):
        self.depth = depth
        self.nnue = NNUE()
        self.nnue.load_state_dict(torch.load('./checkpoint/nnue_1000data_50epochs.pth', weights_only=True))
        self.nnue.eval()
    def evaluate_board(self, board):
        """Evaluate the board position based on material and piece-square tables."""
        if board.is_checkmate():
            return -10000 if board.turn == chess.WHITE else 10000
        if board.is_stalemate() or board.is_insufficient_material():
            return 0
        
        fen = board.fen()
        fen = np.stack(encode_fen_string(fen))
        fen = torch.tensor(fen, dtype=torch.float32).unsqueeze(0)
        score = self.nnue(fen)
        return score

    def alphabeta(self, board, depth, alpha, beta, maximizing_player):
        """Minimax with Alpha-Beta pruning."""
        if depth == 0 or board.is_game_over():
            return self.evaluate_board(board)
        
        if maximizing_player:
            max_eval = float('-inf')
            for move in board.legal_moves:
                board.push(move)
                eval = self.alphabeta(board, depth - 1, alpha, beta, False)
                board.pop()
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break  # Beta cutoff
            return max_eval
        else:
            min_eval = float('inf')
            for move in board.legal_moves:
                board.push(move)
                eval = self.alphabeta(board, depth - 1, alpha, beta, True)
                board.pop()
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break  # Alpha cutoff
            return min_eval

    def find_best_move(self, board):
        """Find the best move for the current position."""
        if board.is_game_over():
            return None
        
        best_move = None
        best_value = float('-inf') if board.turn == chess.WHITE else float('inf')
        alpha = float('-inf')
        beta = float('inf')
        
        for move in board.legal_moves:
            board.push(move)
            value = self.alphabeta(board, self.depth - 1, alpha, beta, not board.turn)
            board.pop()
            
            if board.turn == chess.WHITE:  # Maximizing (White)
                if value > best_value:
                    best_value = value
                    best_move = move
                alpha = max(alpha, value)
            else:  # Minimizing (Black)
                if value < best_value:
                    best_value = value
                    best_move = move
                beta = min(beta, value)
        
        return best_move

In [34]:
chessAI = MinimaxNNUE(depth=10)
board = chess.Board('7k/8/8/5pp1/5p1P/5P2/5K2/8 w - - 0 1')
chessAI.find_best_move(board)

Move.from_uci('f2g2')